### Create Ad Hoc Structure for listing all the required combination Search Parameters

- uses Spreadsheet
- uses set theory and math to enumarate combinations
- uses exclude list to remove prohibited combination
- uses least strong Conformance of members of combot to determine conformance


create a Dict with key = Resource_type and value equal to a sorted list of tuples with combonent parameter list, conformance  e.g. {Type:[(p_combo-1,c1),(p_combo2,c2...]}

add new sheet to spreadsheet for manual review

### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles
- use spreadhsheet as input data  ( could use the sp too )
- get combos from spreadsheet
  - update parameter types if needed
  

In [ ]:
def get_conf_str(combo, r_type):
    conf_str = ''
    for k in df.itertuples(index=True):
        if k.base == r_type and k.code in combo:
            if  k.base_conf == 'MAY' or k.base_conf in none_list:
                conf_str = 'MAY'
                break
            elif k.base_conf == 'SHOULD':
                conf_str = 'SHOULD' 
            elif k.base_conf == 'SHALL' and conf_str not in ['SHALL','MAY']:
                conf_str ='SHALL' 
    return conf_str

def get_type_list(combo, r_type):
    t = {i.type for i in df.itertuples(index=True) if i.code in combo and i.base == r_type}
    print(t)
    return list(t)


def get_combos(pairs,c_list):
    a_list =[]
    for i in c_list:
        for j in c_list:
            #print(f'i={i} j = {j}, i&j= {i&j} i^j= {i^j}')
            if i & j and i != j:
                #print(f'i={i}, j = {j}, i&j= {i&j},i^j= {i^j} i|j = {i|j}')
                if i^j in pairs:
                    
                    if i|j not in a_list + c_list:
                        a_list.append(i|j)
    return a_list

          

combo_list = []
combo_dict = {}
for base in r_type:
    print(base)
    pairs = []
    sp_len = len([d for d in data if d.base== base])
          
    for d in data:
        if d.base == base and d.combo_pairs not in none_list: 
            pairs.extend([{d.code, p} for p in d.combo_pairs.split(',') if p != d.code])  # get allowed pairs
            #print(f'pairs = {pairs}')
          
    combo_list = pairs
    for j in range(sp_len-1):
         combo_list = combo_list + get_combos(pairs,combo_list)
          
    # convert to sorted tuples
    combo_list = [sorted(tuple(i)) for i in combo_list]
    combo_list = sorted(combo_list)
          
    try: #subtract forbidden combos
        f_combos = []
        df_r = read_excel(xls,'resources',na_filter = False)
        for r in df_r.itertuples(index=True):
            if  r.type == base:
                f_combos = r.forbidden_s_combos.split(',')
                #print(f_combos)
                f_combos = [sorted(i.split('|')) for i in f_combos]
                #print(f_combos)
        combo_list = [i for i in combo_list if sorted(i) not in f_combos]
    except AttributeError:
        print('forbidden_s_combos is missing')
          
    print(f'{len(combo_list)} combos = {combo_list}')
    #combo_list.extend(combo_list)
    combo_dict[base]=[(c, get_conf_str(combo = c, r_type = base), get_type_list(combo = c, r_type = base)) for c in combo_list]
# print(dumps(combo_dict, indent = 4))

### Convert combo_dict to dataframe and add to spreadsheet for review

In [ ]:
rows=[]
for k,v in combo_dict.items():
    for i in v:
        rows.append([k, *i])
        
generated_combos= DataFrame(rows, columns = ['Base Type',"Combo",'Conformance','Parameter Types',] )

book = load_workbook(f'{in_path}{in_file}.xlsx')
std = book['generated_combos']  # remove sheet first if  exists
book.remove(std)
#update information
with ExcelWriter(f'{in_path}{in_file}.xlsx', engine = 'openpyxl') as writer:
    writer.book = book
    
    generated_combos.to_excel(writer, sheet_name = 'generated_combos')